In [1]:
import numpy as np
import skimage
from skimage.io import imread_collection
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import cv2
from copy import deepcopy
import glob
import cv2 
import pickle

### Reading Input Files


In [2]:
pathway = "/Users/krishnabairavi/Downloads/train/*.png"

In [3]:
images = [mpimg.imread(file) for file in glob.glob(pathway)]

In [4]:
print (images[0].shape)

(750, 7522, 4)


### Procedure followed 

1. On visual inspection, it can be observed that for every image graph, there are 10 rows and 100 column gridboxes. On visualising the size of the image, the  height and width correspond to approximately  750 X 7500. Thus each gridbox unit can be identified as a box of dimension 75x75. Now, for every image in trainset, I iterate through the gridboxes 

2. The annotated labels present in blue ( in case of non- V) and green color ( in case of V) needs to be indentified as a next step to generate a train set that contains the amplitude of the corresponding peak for both  V and Non-V's. 

3. This amplitude for each peak in a signal is obtained by detecting the black colored signals and using Harris point detectors to find the peak points  

4. The generated amplitutdes and the annotations ( obtained from the color of annotations in figures) are used to creat train dataset where the input is amplitude for every signal peak in an image and output is the classified label ( V or Non-V)

5. Logistic Regression classifier is used for classification 

In [6]:
feature_set = []

for img_number in range(len(images)):
    
    img = cv2.cvtColor(images[img_number], cv2.COLOR_BGRA2BGR)
    total_count = 0
    
  
    for i in range(10,7500,75):
        
        pic = img[0:75,i:i+75]
        flag = 0
        break_flag = 0
        pic_mod = deepcopy(pic)
        pic_mod = pic_mod/np.linalg.norm(pic_mod)
        sum_pic = np.sum(pic_mod[:,:,2])
   
        ## Code block to detect blue regions in the annotations ( the one's which are non V's)
        
        if(sum_pic > 43.5 and sum_pic < 45):
            box_number = 0
            for m in range(75,750,75):
                pic_m = img[m:m+75,i:i+75]
            
                pic_mod_b = cv2.cvtColor(pic_m, cv2.COLOR_BGR2GRAY)
                sum_pic_b = np.sum(pic_mod_b)
                
                box_number += 1
                
                ## To find the black peak in an image using color identification and Harris peak detections
                
                if(sum_pic_b < 5450):
                    
                    total_count += 1  
                    
                    operatedImage = np.float32(pic_mod_b)
            
                    dst = cv2.cornerHarris(operatedImage, 2, 5, 0.01)
                    
                    dst_new = cv2.dilate(dst,None)

                    points=np.unravel_index(dst_new.argmax(),dst.shape)

                    append_val_blue = (((4 - box_number)*75) +points[0])*0.5 

                    ## Making it a  binary classification problem
                    
                    ## Blue color represents "non-V", which should be determined as class 0
                    
                    if(append_val_blue < 60):
                        feature_set.append( (0, abs((((4 - box_number)*75)+points[0])*0.5 )))
                    break

                    
        ## Code block to detect blue regions in the annotations ( the one's which are non V's)
    
        if(sum_pic > 42 and sum_pic < 43):
            box_number_green = 0

        ## To find the black peak in an image using color identification and Harris peak detections
            
            for m_green in range(75,750,75):
                pic_m_green = img[m_green:m_green+75,i:i+75]
                pic_mod_b_green = cv2.cvtColor(pic_m_green, cv2.COLOR_BGR2GRAY)
                sum_pic_b_green = np.sum(pic_mod_b_green)
                box_number_green += 1

                if(sum_pic_b_green < 5450):                    
            
                    operatedImage_green = np.float32(pic_mod_b_green)
            
                    dst_green = cv2.cornerHarris(operatedImage_green, 2, 5, 0.01)
                    
                    dst_new_green = cv2.dilate(dst_green,None)

                    points_new_green = np.unravel_index(dst_new_green.argmax(),dst_green.shape)

                    append_val =  abs((((4 - box_number_green)*75) +points_new_green[0])*0.5 )
                    
                    ## Green color represents "V", which should be determined as class 1
                    
                    if(abs(append_val) > 60):
                        feature_set.append((1, abs((((4 - box_number_green)*75) +points_new_green[0])*0.5 )))
                    break
   

The feature_set is a list of tuples, where each tuple represents a class and it's signal peak's amplitude 

In [9]:
## Splitting the dataset into train and val sets. X-list contains set of amplitudes for every peak and  Y list contains the binary class of the signal ( 0 and 1)

x_list=[]
y_list=[]
for i in feature_set:
    y_list.append(i[0])
    x_list.append(i[1])

x_list = np.asarray(x_list)
y_list = np.asarray(y_list)

In [10]:
x_list = np.reshape(x_list, (-1,1))
y_list = np.reshape(y_list, (-1,1))

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x_list,y_list, test_size=0.1, random_state=0)

 ### Using Logistic Regression model for classification 

In [13]:
logmodel = LogisticRegression(max_iter=500, tol=0.000001)
logmodel.fit(x_train, y_train)

/Users/krishnabairavi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/krishnabairavi/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=1e-06, verbose=0,
                   warm_start=False)

### Saving the model to file for retreiving it during testing phase 

In [ ]:
filename = 'fin.sav'
pickle.dump(logmodel, open(filename, 'wb'))